In [19]:
import numpy as np
import pandas as pd
#import skdim
from scipy.linalg import eigh
from sklearn.preprocessing import scale
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [17]:
foreground = np.load("foreground.npy")
background = np.load("background.npy")
flabels = np.load("foreground_labels.npy")

In [4]:
print(foreground.shape)
print(background.shape)
print(foreground[:3, :3])
print(background[:3, :3])

(5000, 784)
(5000, 784)
[[0.44313725 0.59215686 0.59215686]
 [0.44313725 0.44705882 0.44313725]
 [0.39607843 0.20392157 0.18431373]]
[[0.45882353 0.56470588 0.5372549 ]
 [0.75294118 0.75686275 0.75686275]
 [0.51372549 0.45490196 0.44313725]]


In [9]:
Sigma1 = np.cov(foreground.T)

In [10]:
Sigma2 = np.cov(background.T)

In [14]:
eigenvalues, eigenvectors = np.linalg.eig(Sigma1 - Sigma2)
sorted_indices = np.argsort(eigenvalues)[::-1][:10]
top_10_eigenvectors = eigenvectors[:, sorted_indices]


In [16]:
top_10_eigenvectors.shape

(784, 10)

In [21]:
X = foreground @ top_10_eigenvectors[:,:1]

In [22]:
classifier = LogisticRegression()
scores = cross_val_score(classifier, X, flabels, 
                         cv=5, scoring='accuracy')

In [23]:
mean_accuracy = scores.mean()
std_accuracy = scores.std()

In [24]:
mean_accuracy

0.583

In [25]:
std_accuracy

0.0061967733539318726

In [26]:
stats = np.zeros((2, 10))

In [28]:
np.random.seed(42)
for i in range(10):
    X = foreground @ top_10_eigenvectors[:,:i+1]
    classifier = LogisticRegression()
    scores = cross_val_score(classifier, X, flabels, 
                         cv=5, scoring='accuracy')
    stats[0, i] = scores.mean()
    stats[1, i] = scores.std()

In [29]:
stats

array([[0.583     , 0.9414    , 0.941     , 0.9432    , 0.9454    ,
        0.9472    , 0.9464    , 0.9468    , 0.9474    , 0.9472    ],
       [0.00619677, 0.00531413, 0.00576194, 0.00614492, 0.00674092,
        0.00658483, 0.00731027, 0.00738647, 0.00662118, 0.00676461]])